In [2]:
%pip install -q cassio datasets langchain openai tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

c:\Users\manun\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%pip install PyPDF2

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     -------------------------------------- 232.6/232.6 kB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from PyPDF2 import PdfReader

In [1]:
ASTRA_DB_APPLICATION_TOKEN =  "AstraCS:vZPjBkdaZnmqrOTF-----------------------bd50cb048a26fc7b7d"
ASTRA_DB_ID = 'fb1d810a-a-------------------e0'

OPENAI_API_KEY = 'sk-tybb3Z04j8bIiO7-------------------k8'

In [4]:
pdf = PdfReader("finance.pdf")

In [5]:
from typing_extensions import Concatenate

text = ""

for page in pdf.pages:
    content = page.extract_text()
    if content:
        text += content



In [6]:
text

"Chapter 1\nIntroduction to Finance\nRoad Map\nPart A Introduction to ﬁnance.\n•Financial decisions and ﬁnancial markets.\n•Present value.\nPart B Valuation of assets, given discount rates.\nPart C Determination of risk-adjusted discount rates.\nPart D Introduction to derivatives.\nMain Issues\n•What Is Finance\n•Valuation of Assets\n–Opportunity Cost of Capital\n–Present Value (PV)\n•Role of Financial Markets\n•Objectives of Financial ManagerChapter 1 Introduction to Finance 1-1\n1 What is Finance?\n•Finance is about the bottom line of business activities.\n•Every business is a process of acquiring and disposing assets:\n–Real assets (tangible and intangible).\n–Financial assets.\n•Two objectives of business:\n–Grow wealth.\n–Use wealth (assets) to best meet economic needs.\n•Financially, a business decision reduces to valuation of assets.\n•Valuation is the central issue of ﬁnance.\nFall 2006 c/circlecopyrtJ. Wang 15.401 Lecture Notes1-2 Introduction to Finance Chapter 1\nQuestions w

In [9]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [10]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

c:\Users\manun\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\manun\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
vector_store = Cassandra(
    embedding=embedding,
    table_name='vector_store',
    session=None,
    keyspace=None
)

In [12]:
from langchain.text_splitter import CharacterTextSplitter

splitted_text = CharacterTextSplitter(
    separator="\n",
    chunk_size = 800,
    chunk_overlap =200,
    length_function = len
)

text = splitted_text.split_text(text)

In [13]:
vector_store.add_texts(text[:50])
vector_index = VectorStoreIndexWrapper(vectorstore=vector_store)

In [17]:
first = True
while True:
    if first:
        ques = input("Enter your question (type q to exit)")
    else:
        ques = input("What's your next question (type q to exit)")
    if ques.lower() == "q":
        break
    if ques == "":
        continue

    first = False

    print("\nQuestion : \"%s\""% ques)
    answer = vector_index.query(ques,llm=llm).strip()

    print("\nAnswer : \"%s\""% answer)    
    



Question : "what's the two objective of business"

Answer : "The two objectives of business are to grow wealth and to use wealth to best meet economic needs."

Question : "what is a business"

Answer : "A business is an organization or enterprise that engages in commercial, industrial, or professional activities with the goal of generating profit. It typically involves the production or sale of goods or services to customers in exchange for money. A business can take many forms, such as a sole proprietorship, partnership, corporation, or non-profit organization, and can operate in various industries and sectors of the economy."
